# QTM350 - Group Project
## Group members

Please find the names of the group members below:

In [1]:
# Import packages
import sqlite3
import pandas as pd
import random

In [2]:
# Connect to the database
connection = sqlite3.connect("groups.db")
cursor = connection.cursor()

In [3]:
# Execute the SQL commands to create the table
cursor.execute('''
DROP TABLE IF EXISTS groups;
''')

cursor.execute('''
CREATE TABLE groups (
    student_id INTEGER PRIMARY KEY AUTOINCREMENT,
    student_name TEXT NOT NULL UNIQUE,
    emory_id INT UNIQUE, -- Allow NULL and ensure uniqueness
    group_id INT
);
''')

# Commit the changes for table creation
connection.commit()

In [4]:
# Insert data into the table
insert_sql = '''
INSERT INTO groups (student_name, emory_id, group_id) VALUES (?, ?, ?)
'''

# List of tuples: (student_name, emory_id, group_id)
# Using Python None here, as executemany handles it correctly for sqlite3
students_data = [
    ('Ahshar Brown', 2575182, None),
    ('Kimberly Cardinale ', 2548934, 1),
    ('Maisy Christians', 2550529, None),
    ('Jennifer Gu', 2606428, None),
    ('Alan Na', 2588698, None),
    ('Carl Yu', 2550364, 1)
]

cursor.executemany(insert_sql, students_data)

# Commit the transaction for data insertion
connection.commit()

In [5]:
# --- Random Group Assignment Logic ---
print("\n--- Assigning Remaining Students to Groups ---")

# Ensure connection is still open or reconnect
try:
    # Check if cursor is usable by executing a simple query
    cursor.execute("SELECT 1") 
except (sqlite3.ProgrammingError, AttributeError, NameError): 
    print("Reconnecting to database for group assignment...")
    connection = sqlite3.connect(db_file)
    cursor = connection.cursor()

# Fetch unassigned students (student_id)
cursor.execute("SELECT student_id FROM groups WHERE group_id IS NULL")
unassigned_rows = cursor.fetchall()
unassigned_ids = [row[0] for row in unassigned_rows] # Extract IDs
n = len(unassigned_ids)

print(f"Found {n} unassigned students.")

g2 = 0 # Number of groups of 2
g3 = 0 # Number of groups of 3
possible = True

if n >= 2:
    # Calculate number of groups of 2 and 3, prioritising groups of 2
    if n % 2 == 0:
        # Even number - all groups of 2
        g2 = n // 2
        g3 = 0
    else:
        # Odd number - need at least one group of 3
        if n >= 3:
            g2 = (n - 3) // 2  # Form one group of 3, rest groups of 2
            g3 = 1
        else:
            # Only 1 student left, cannot form any group
            possible = False
            print(f"Cannot form groups with only {n} student(s). Need at least 2 students.")

    if possible:
        print(f"Creating {g2} groups of 2 and {g3} groups of 3.")

        # Shuffle the student IDs randomly
        random.shuffle(unassigned_ids)

        # Find the next available group ID
        cursor.execute("SELECT MAX(group_id) FROM groups WHERE group_id IS NOT NULL")
        max_existing_group_id_result = cursor.fetchone()
        # Handle case where there are no existing groups (max_existing_group_id_result[0] would be None)
        max_existing_group_id = max_existing_group_id_result[0] if max_existing_group_id_result and max_existing_group_id_result[0] is not None else 0
        next_group_id = max_existing_group_id + 1 
        print(f"Starting new groups from ID: {next_group_id}")

        current_student_index = 0
        update_sql = "UPDATE groups SET group_id = ? WHERE student_id = ?"

        # Assign groups of 2
        print("\nAssigning groups of 2...")
        for i in range(g2):
            group_members_ids = unassigned_ids[current_student_index : current_student_index + 2]
            if group_members_ids: # Ensure there are members to assign
                print(f"  Assigning group {next_group_id} to students (IDs): {group_members_ids}")
                for student_id in group_members_ids:
                    cursor.execute(update_sql, (next_group_id, student_id))
                next_group_id += 1
                current_student_index += 2
            else:
                print("  Warning: Tried to assign an empty group of 2.")

        # Assign groups of 3
        print("\nAssigning groups of 3...")
        for i in range(g3):
            group_members_ids = unassigned_ids[current_student_index : current_student_index + 3]
            if group_members_ids: # Ensure there are members to assign
                print(f"  Assigning group {next_group_id} to students (IDs): {group_members_ids}")
                for student_id in group_members_ids:
                    cursor.execute(update_sql, (next_group_id, student_id))
                next_group_id += 1
                current_student_index += 3
            else:
                 print("  Warning: Tried to assign an empty group of 3.")

        # Commit the updates for group assignments
        connection.commit()
        print("\nFinished assigning groups.")

else:
    print("Not enough unassigned students (need at least 2) to form new groups.")

# The final query cell will display the results.
print("--- End of Group Assignment ---\n")


--- Assigning Remaining Students to Groups ---
Found 4 unassigned students.
Creating 2 groups of 2 and 0 groups of 3.
Starting new groups from ID: 2

Assigning groups of 2...
  Assigning group 2 to students (IDs): [1, 3]
  Assigning group 3 to students (IDs): [5, 4]

Assigning groups of 3...

Finished assigning groups.
--- End of Group Assignment ---



In [6]:
# Query the table and display results using pandas
query = '''
SELECT * FROM groups
ORDER BY group_id, student_name
'''
# Reconnect if connection was closed, or ensure it's open
connection = sqlite3.connect("groups.db")
df = pd.read_sql_query(query, connection)

# Close the connection
connection.close()

# Display the DataFrame
df

,student_id,student_name,emory_id,group_id
0,6,Carl Yu,2550364,1
1,2,Kimberly Cardinale,2548934,1
2,1,Ahshar Brown,2575182,2
3,3,Maisy Christians,2550529,2
4,5,Alan Na,2588698,3
5,4,Jennifer Gu,2606428,3
